In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar  5 15:32:28 2019

@author: hyunseok
"""

#module importing
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# 2. driver경로설정 및 url주소 설정
driverpath = 'C:/Users/HS/Desktop/driver/chromedriver.exe'
dr = webdriver.Chrome(driverpath)

####

#로그인
dr.get('https://www.kita.net/login/login.do?geturl=http://stat.kita.net/stat/pmsd/china/ChinaWholeList.screen')
dr.find_element_by_id('p_member_id').send_keys('kedkorea12')
dr.find_element_by_id('p_passwd').send_keys('ked2363!')
dr.find_element_by_xpath('''//*[@id="myform"]/fieldset/button''').click()

####

df_tot = pd.DataFrame(columns = ['year','month','country','LC','MC','SC','S_code', 'export','export_rate', 'import','import_rate','CA'])

#중일EU미국 URL
naraURL = ['http://stat.kita.net/stat/pmsd/china/ChinaWholeList.screen']
nara = ['중국']
for site, 나라 in zip(naraURL, nara) :
    #사이트 진입
    dr.get(site)
    #품목별 클릭
    dr.find_element_by_xpath('''//*[@id="contents"]/div[1]/div[1]/ul/li[2]/a''').click()
    time.sleep(2)
    #누계 -> 당월
    dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[3]/select/option[1]''').click()

    #품목 대분류 선택
    for item_1 in range(1,13):
    #for item_1 in [4]:
        dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[1]/div[1]/select[1]/option[{0}]'''.format(item_1)).click()
        대분류 = dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[1]/div[1]/select[1]/option[{0}]'''.format(item_1)).text
        
        #품목 중분류 선택
        tmp=2
        while 1:
            try:
                dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[1]/div[1]/select[2]/option[{0}]'''.format(tmp)).click()
                중분류 = dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[1]/div[1]/select[2]/option[{0}]'''.format(tmp)).text
                tmp +=1
            except:
                break;
                
            #년월 Trick
            dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(4)).click()
            dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(5)).click()   
            dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[3]/a''').click()
            time.sleep(2)
            
            #년 월
            for year_ in range(1, 18):
                dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(year_)).click()
                년 = int(dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[1]/select/option[{0}]'''.format(year_)).text)

                for month_ in range(1,13):
                    dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(month_)).click()
                    월 = int(dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[2]/select/option[{0}]'''.format(month_)).text)
                    
                    
                    #조회
                    dr.find_element_by_xpath('''//*[@id="contents"]/div[2]/form/fieldset/div[3]/a''').click()                    
                    time.sleep(0.5)
                    drt = dr.page_source
                    soup = BeautifulSoup(drt,'html.parser')

                    #데이터 적재
                    코드_소 = []
                    소분류 = []
                    수출액 = []
                    수출증감률 = []
                    수입액 = []
                    수입증감률 = []
                    수지 = []

                    i=1
                    for cycle in range(int((len(soup.findAll('td', {'align' : 'Left'}))-1)/2)) :
                        코드_소.append(soup.findAll('td', {'align' : 'Center'})[-i+1].text)
                        소분류.append(soup.findAll('td', {'align' : 'Left'})[i*2].text)
                        수출액.append(soup.findAll('td', {'align' : 'Right'})[i*10+5].text)
                        수출증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+6].text)
                        수입액.append(soup.findAll('td', {'align' : 'Right'})[i*10+7].text)
                        수입증감률.append(soup.findAll('td', {'align' : 'Right'})[i*10+8].text)
                        수지.append(soup.findAll('td', {'align' : 'Right'})[i*10+9].text)
                        i+=1

                    df = pd.DataFrame({'year' :년, 'month' :월, 'country':나라, 'LC':대분류, 'MC':중분류, 'SC':소분류, 'S_code':코드_소, 'export':수출액, 'export_rate':수출증감률, 'import':수입액, 'import_rate':수입증감률, 'CA':수지})
                    df_tot = pd.concat([df_tot, df])

                    print(df.shape)

df_tot.to_csv('./df_tot.csv',index=False)

(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)


(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)
(4, 12)


(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)


(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)
(2, 12)


(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)


...

(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)
(6, 12)


(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)


(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)


(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)


(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
(1, 12)
